## CMSC 35300 Final Project

In [1]:
import numpy as np
from scipy.sparse import csc_matrix
from pandas import read_csv

In [16]:
k = 10
lambd = 4

In [4]:
df = read_csv('ml-25m/ratings.csv')
df = df.drop('timestamp', axis=1)
df.loc[:,'rating'] *= 2
df = df.astype(int)

max_userId = df.max().loc['userId']
max_movieId = df.max().loc['movieId']

In [9]:
print(max_movieId)
print(max_userId)

209171
162541


In [7]:
userId_to_movieId_list = {}
for row in df.itertuples(): # In df, userId and movieId starts from 1
    # make userId and movieId start from 0
    if row.userId-1 not in userId_to_movieId_list.keys():
        userId_to_movieId_list[row.userId-1]=[row.movieId-1]
    else:
        userId_to_movieId_list[row.userId-1].append(row.movieId-1)

movieId_to_userId_list = {}
for row in df.itertuples(): # In df, userId and movieId starts from 1
    # make userId and movieId start from 0
    if row.movieId-1 not in movieId_to_userId_list.keys():
        movieId_to_userId_list[row.movieId-1]=[row.userId-1]
    else:
        movieId_to_userId_list[row.movieId-1].append(row.userId-1)

In [13]:
ratings = {}
for row in df.itertuples():
    # make userId and movieId start from 0
    ratings[(row.userId-1, row.movieId-1)]=row.rating

In [22]:
X = np.random.uniform(0,10,(k, max_userId))
Y = np.random.uniform(0,10,(k, max_movieId))
X1 = np.zeros((k,max_userId))
Y1 = np.zeros((k,max_movieId))

In [18]:
import time
startTime = time.time()

for u in range(max_userId): # u starts from 0
    if u not in userId_to_movieId_list.keys(): # in this dictionary, userId starts from 0
        continue
    sum_yi_yiT = np.zeros((k,k))
    sum_rui_yi = np.zeros((k,1))
    for i in userId_to_movieId_list[u]: # in this dictionary, userId and movieId starts from 0
        sum_yi_yiT += Y[:,i:i+1]@Y[:,i:i+1].T
        sum_rui_yi += ratings[(u,i)]*Y[:,i:i+1]
    X1[:,u:u+1] = np.linalg.inv(sum_yi_yiT+lambd*np.diag(np.ones(k)))@sum_rui_yi

executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))

Execution time in seconds: 214.7732572555542


In [21]:
import time
startTime = time.time()

for i in range(max_movieId): # i starts from 0
    if i not in movieId_to_userId_list.keys():
        continue
    sum_xu_xuT = np.zeros((k,k))
    sum_rui_xu = np.zeros((k,1))
    for u in movieId_to_userId_list[i]: # in this dictionary, userId and movieId starts from 0
        sum_xu_xuT += X[:,u:u+1]@X[:,u:u+1].T
        sum_rui_xu += ratings[(u,i)]*X[:,u:u+1]
    Y1[:,i:i+1] = np.linalg.inv(sum_xu_xuT+lambd*np.diag(np.ones(k)))@sum_rui_xu

executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))

Execution time in seconds: 268.2666640281677


In [ ]:
import time
startTime = time.time()

iteration = 0
while True:
    iteration += 1
    print('Current Iteration: ' + str(iteration))
    
    for u in range(max_userId): # u starts from 0
        if u not in userId_to_movieId_list.keys(): # in this dictionary, userId starts from 0
            continue
        sum_yi_yiT = np.zeros((k,k))
        sum_rui_yi = np.zeros((k,1))
        for i in userId_to_movieId_list[u]: # in this dictionary, userId and movieId starts from 0
            sum_yi_yiT += Y[:,i:i+1]@Y[:,i:i+1].T
            sum_rui_yi += ratings[(u,i)]*Y[:,i:i+1]
        X1[:,u:u+1] = np.linalg.inv(sum_yi_yiT+lambd*np.diag(np.ones(k)))@sum_rui_yi
    
    for i in range(max_movieId): # i starts from 0
        if i not in movieId_to_userId_list.keys():
            continue
        sum_xu_xuT = np.zeros((k,k))
        sum_rui_xu = np.zeros((k,1))
        for u in movieId_to_userId_list[i]: # in this dictionary, userId and movieId starts from 0
            sum_xu_xuT += X[:,u:u+1]@X[:,u:u+1].T
            sum_rui_xu += ratings[(u,i)]*X[:,u:u+1]
        Y1[:,i:i+1] = np.linalg.inv(sum_xu_xuT+lambd*np.diag(np.ones(k)))@sum_rui_xu

    if np.linalg.norm(X1-X)<1e-5 and np.linalg.norm(Y1-Y)<1e-5:
        break
    
    X = X1
    Y = Y1
        
executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))
print('Total iterations: ' + str(iteration))

Current Iteration: 1
